In [1]:
from __future__ import division

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Problem 4

In [3]:
from scipy.io import loadmat
spam = loadmat('./homework2/data/spam_fixed.mat')

In [4]:
spam_data = spam['data']
spam_labels = spam['labels']
spam_test_data = spam['testdata']
spam_test_labels = spam['testlabels']

In [116]:
def compute_errors(predictions, labels):
    """ Generates a list of indexes of misclassified 
    examples
    """
    zipped = zip(predictions, labels)
    errors = [ix for ix, tup in enumerate(zipped)
              if tup[0] != tup[1]]
    
    return errors

In [117]:
def test_scikits_model(clf, test_data, test_labels):
    preds = clf.predict(test_data)
    errors = compute_errors(preds, [t[0] for t in test_labels])

    return len(errors) / test_labels.shape[0]

In [118]:
def combinations(x):
    combos = np.array(list(itertools.combinations(x,2)))
    x_prime = np.array([x1 * x2.T for x1, x2 in combos])
    
    return x_prime

In [119]:
def expand_features(features):
    original = features
    squared = features ** 2
    combos = combinations(features)
    return np.concatenate([original, squared, combos])

### 4.1 Averaged-Perceptron with 64 passes through the data.

In [100]:
def predict(features,  weights):
    """ 
    Predicts a label (1, -1) given a vector 
    of features and weights
    
    Args:
        features:
        weights:

    Return:
        prediction: int of -1 or 1
    """
    prediction = np.dot(features, weights)
    if prediction > 0:
        return 1
    else:
        return -1

In [101]:
def update_weights(prediction, label, features, weights):
    """
    Args:
        prediction: int of predicted label (1 or -1)
        label: the true label of the data point (1 or -1)
        features: numpy array of feature values
        weights: 1d numpy array of weights for the features
    
    Returns:
        weights: 
    """
    if prediction != label:
        weights = weights + (label * features)
    
    return weights

In [102]:
def perceptron_fit(examples):
    """ 
    Generates a vector of weights
    
    Args:
        examples: vector of feature, label tuples
    
    Returns:
        weights: d-dimensional vector of weights
    """
    weights = np.zeros(examples[0][0].shape)
    for features, label in examples:
        prediction = predict(features, weights)
        weights = update_weights(prediction, label[0], 
                                 features, weights)
    return weights

In [111]:
# add trained bias to signature and prediction
def test_perceptron_model(predict, testdata, testlabels, trained_weights):
    """ Generates predictions from a trained weight vector """
    preds = [predict(features, trained_weights)
             for features in testdata]
    errors = compute_errors(preds, [t[0] for t in testlabels])
    
    return len(errors) / testlabels.shape[0]

In [104]:
def avg_perceptron_train(num_iterations, examples):
    weights = np.zeros(examples[0][0].shape)
    cweights = np.zeros(examples[0][0].shape)
    bias = 0
    cbias = 0
    counter = 1
    
    for iteration in range(0, num_iterations):
        np.random.shuffle(examples)
        for features, label in examples:
            if np.dot(features, weights) + bias <= 0:
                # update the weights for this iteration
                weights = weights + (label * features)
                bias = bias + label
                # update the cached weights
                cweights = cweights + (label * counter * features)
                cbias = bias + (label * counter)
            counter += 1
            
    return (weights - ((1/counter) * cweights), bias - ((1/counter) * cbias))

In [105]:
def avg_perceptron_train(num_iterations, examples):
    weights = np.zeros(examples[0][0].shape)
    cweights = np.zeros(examples[0][0].shape)
    bias = 0
    counter = 1
    
    for iteration in range(0, num_iterations):
        np.random.shuffle(examples)
        for features, label in examples:
            if np.dot(features, weights) <= 0:
                # update the weights for this iteration
                weights = weights + (label * features)
                # update the cached weights
                cweights = cweights + (label * counter * features)
            counter += 1
            
    return weights - ((1/counter) * cweights)

In [106]:
def avg_perceptron_test(features, weights, bias):
    activation = np.dot(features, weights) + bias
    if activation > 0:
        return 1
    else:
        return -1

In [121]:
# 1. Averaged-Perceptron with 64 passes through the data.
spam_examples = zip(spam_data, spam_labels)
trained_weights = avg_perceptron_train(64, spam_examples)
test_perceptron_model(predict, spam_test_data, spam_test_labels, trained_weights)

0.14583333333333334

### 4.2 Logistic regression model with MLE for parameter estimation.

In [124]:
from sklearn.linear_model import LogisticRegression

In [125]:
clf = LogisticRegression()
clf.fit(spam_data, np.ravel(spam_labels))
test_scikits_model(clf, spam_test_data, spam_test_labels)

0.07747395833333333

### 4.3  Generative model classifier where class conditional distributions are multivariate Gaussian distributions with shared covariance matrix for all classes. Use MLE for parameter estimation.

In [24]:
# http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(spam_data, np.ravel(spam_labels))
test_scikits_model(lda, spam_test_data, spam_test_labels)

0.12239583333333333

### 4.4 Same as above, except arbitrary Gaussians (i.e., each class with its own covariance matrix).

In [21]:
# http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(spam_data, np.ravel(spam_labels))
test_scikits_model(qda, spam_test_data, spam_test_labels)

0.17447916666666666

### 4.5 Averaged Percepton w/ Feature Map

In [99]:
spam_expanded = np.array([expand_features(x) for x in spam_data])

In [114]:
spam_test_expanded = np.array([expand_features(x) for x in spam_test_data])

In [108]:
expanded_examples = zip(spam_expanded, spam_labels)

In [120]:
trained_weights = avg_perceptron_train(64, expanded_examples)
test_perceptron_model(predict, spam_test_expanded, spam_test_labels, trained_weights)

0.6126302083333334

### 4.6 Logistic Regression w/ Feature Map

In [127]:
clf = LogisticRegression()
clf.fit(spam_expanded, np.ravel(spam_labels))
test_scikits_model(clf, spam_test_expanded, spam_test_labels)

0.07747395833333333